# WALLABY internal data access download

A notebook pre-filled with cells and scripts for downloading WALLABY internal release data. The notebook has the following sections

1. Authentication
2. Get source finding catalog and products
3. Get kinematic model table and products

---

In [1]:
import os
import getpass
import requests
import getpass
import pyvo as vo
from pyvo.auth import authsession, securitymethods
from astropy.io.votable import from_table, parse_single_table

# 1. Connect

We access the internally released WALLABY via TAP. The link to the tap service is provided below. The password to the `wallaby_user` will be circulated internally by the WALLABY project team, and it will be required to access any of the data.

### Authenticate

<span style="font-weight: bold; color: #FF0000;">⚠ Update the cell below with your username and enter your password</span>

In [2]:
# Enter WALLABY user username and password

username = 'wallaby_user'
password = getpass.getpass('Enter your password')

Enter your password········


In [3]:
# Connect with TAP service

URL = "https://wallaby.aussrc.org/tap"
auth = vo.auth.AuthSession()
auth.add_security_method_for_url(URL, vo.auth.securitymethods.BASIC)
auth.credentials.set_password(username, password)
tap = vo.dal.TAPService(URL, session=auth)

# 2. Source finding

First we need to identify which internal release we want to access. The WALLABY team uses tags to classify source finding detections as internally released. You can view all of the tags by running the cells below. Then, we set the `tag_name` variable two cells below. This will be used later in the notebook, so once you know what data you would like to access, update this value accordingly.

In [4]:
# Get all tags

query = "SELECT * FROM wallaby.tag"
votable = tap.search(query)
table = votable.to_table()
table

id,name,description,added_at
int64,object,object,object
2,Norma DR1,Norma DR1 data release,2022-02-17T00:49:41.748948
3,NGC 5044 DR1,NGC 5044 DR1 data release,2022-02-17T05:20:01.326107
6,Hydra DR1,Hydra DR1 data release,2022-02-23T04:49:08.557440
7,Hydra DR2,Hydra DR2 data release,2022-02-23T04:49:22.070213
12,NGC 5044 DR3,NGC 5044 DR3 data release,2022-10-24T08:44:05.774250
10,NGC 5044 DR2,NGC 5044 DR2 data release,2022-06-10T08:55:44.842245
9,NGC 4808 DR1,NGC 4808 DR1 data release,2022-05-27T08:18:00.781334
8,NGC 4636 DR1,NGC 4636 DR1 data release,2022-02-23T04:49:39.754897
14,Internal Release,The source and corresponding detection have been released internally to the team.,2022-12-02T05:53:15.478050


<span style="font-weight: bold; color: #FF0000;">⚠ Update the `tag_name` value here</span>

In [5]:
# SELECT TAG

tag_name = "NGC 5044 DR1"

In [6]:
# Retrieve catalog as Astropy table

query = """SELECT d.*, ivo_string_agg(t.name || ': ' || t.description, '; ') AS tags, ivo_string_agg(c.comment, '; ') AS comments
        FROM wallaby.detection d
        LEFT JOIN wallaby.tag_detection td ON d.id = td.detection_id 
        LEFT JOIN wallaby.tag t ON t.id = td.tag_id
        LEFT JOIN wallaby.comment c ON d.id = c.detection_id
        WHERE t.name IN ('Internal Data Release', '$TAG_NAME')
        GROUP BY d.id"""
query = query.replace('$TAG_NAME', tag_name)
result = tap.search(query)
table = result.to_table()
table

id,name,run_id,instance_id,access_url,access_format,source_name,x,y,z,x_min,x_max,y_min,y_max,z_min,z_max,n_pix,f_min,f_max,f_sum,rel,flag,rms,w20,w50,ell_maj,ell_min,ell_pa,ell3s_maj,ell3s_min,ell3s_pa,kin_pa,err_x,err_y,err_z,err_f_sum,ra,dec,freq,l,b,v_rad,v_opt,v_app,tags,comments
,,,,,,,pix,pix,pix,pix,pix,pix,pix,pix,pix,,Jy / beam,Jy / beam,Hz Jy,,,Jy / beam,Hz,Hz,pix,pix,deg,pix,pix,deg,deg,pix,pix,pix,Hz Jy,deg,deg,Hz,deg,deg,m / s,m / s,m / s,,
int64,object,int64,int64,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object
4682,SoFiA J134432.82-244925.4,86,167,https://wallaby.aussrc.org/survey/vo/dl/dlmeta?ID=4682,application/x-votable+xml;content=datalink,WALLABY J134432-244925,892.2192722974343,828.6910794180917,5930.480420466962,584.0,605.0,516.0,539.0,722.0,762.0,5492.0,-0.008456071838736534,0.01184215117245913,4962.708632666648,0.9982983729971763,0,0.002604982655527896,575810.2416679957,425258.4382175294,8.044182715264228,5.801783566052931,-9.205918086063775,5.923879573502616,4.761718187956442,-17.19364722750088,137.7057788195698,0.2704957238534749,0.3529004099740369,0.6126948158876361,669.985827289204,206.1367425362275,-24.82372652760987,1405323711.490019,--,--,--,--,--,NGC 5044 DR1: NGC 5044 DR1 data release,
4683,SoFiA J134414.08-231547.6,86,167,https://wallaby.aussrc.org/survey/vo/dl/dlmeta?ID=4683,application/x-votable+xml;content=datalink,WALLABY J134414-231547,912.7966375267697,1765.380706387507,5850.886035304528,608.0,618.0,1459.0,1474.0,653.0,674.0,1570.0,-0.005550565663725138,0.005026679951697588,858.3479820249023,0.9670331513514654,0,0.002414710355013335,305803.3007435854,292091.9646533701,6.154688079922152,4.430776380504492,-20.44248192845933,0.0,0.0,0.0,-1.0,0.4981209955259176,0.6571609308369059,1.19492230125207,332.055276869292,206.0586737670744,-23.26321527118279,1403849741.39442,--,--,--,--,--,NGC 5044 DR1: NGC 5044 DR1 data release,
4686,SoFiA J134002.99-252831.1,86,167,https://wallaby.aussrc.org/survey/vo/dl/dlmeta?ID=4686,application/x-votable+xml;content=datalink,WALLABY J134002-252831,1510.219285286971,449.6839309740759,5450.948663002661,1197.0,1223.0,137.0,162.0,220.0,308.0,12088.0,-0.02482357807457447,0.03819903358817101,28043.9008702014,0.9999999975533298,0,0.007561451779130322,1379746.581870909,1197784.306285804,10.23336711287187,7.073002847152249,-55.90017703864724,11.33521688663817,10.16133348083117,83.44766682365,295.3845286580857,0.2678909318396133,0.2450701235191281,1.314342289480209,2885.213317420493,205.012446592053,-25.47530390468324,1396443493.759208,--,--,--,--,--,NGC 5044 DR1: NGC 5044 DR1 data release,
4689,SoFiA J134321.17-251560.0,86,167,https://wallaby.aussrc.org/survey/vo/dl/dlmeta?ID=4689,application/x-votable+xml;content=datalink,WALLABY J134321-251560,1060.552281782639,566.7393706726875,5375.146685287778,740.0,776.0,253.0,279.0,154.0,220.0,16399.0,-0.02024710923433304,0.02464260533452034,22517.83897053215,0.9999999999710679,0,0.005048318670086136,921515.8572879418,855012.4069108561,19.07482073732793,6.461294385356203,-75.48852433620664,19.82546843029606,4.674523655349004,-75.73354263673336,109.5109592769523,0.468515461222678,0.2205810563135068,0.767903659358751,2243.62856155457,205.8382038629939,-25.26665875033205,1395039753.431156,--,--,--,--,--,NGC 5044 DR1: NGC 5044 DR1 data release,
4690,SoFiA J134255.03-250726.7,86,167,https://wallaby.aussrc.org/survey/vo/dl/dlmeta?ID=4690,application/x-votable+xml;content=datalink,WALLABY J134255-250726,1117.6818226052126,653.3700870157163,5366.6332604437275,808.0,828.0,345.0,361.0,144.0,217.0,7509.0,-0.01123945042490959,0.01789263635873795,9954.584601798539,0.9999981425157802,0,0.00357455217083513,950721.5222104554,856088.9992852381,7.260116910795796,4.7019900

## Download catalog

It is convenient to write the catalog (`astropy` Table object) to a number of file formats. Below we show how to export the table as a `.xml` votable file. For this, we convert the astropy table to a votable, but there are a number of write options for the astropy table:

https://docs.astropy.org/en/stable/io/ascii/write.html

In [7]:
# Download catalog table

votable = from_table(table)
votable_filename = f'{tag_name}.xml'
votable.to_xml(votable_filename)

## Download products

### Download

We have written a function to download the WALLABY source products for an astropy table containing a list of detections. Here the arguments are:

* `table`: the astropy table of detections for which you would like to download product files
* `directory`: the write directory for the products
* `chunk_size`: size (bytes) of each chunk while streaming the download [default 8192 B]

In [8]:
# useful function for downloading table products (requires authentication)

def download_products(row, products_filename, chunk_size=8192):
    """Download products for a row of the table (a detection entry)
    
    """
    name = row['source_name']
    access_url = row['access_url']
    votable = parse_single_table(access_url)
    product_table = votable.to_table()
    url = product_table[product_table['description'] == 'SoFiA-2 Detection Products'][0]['access_url']
    with requests.get(url, auth=(username, password), stream=True) as r:
        r.raise_for_status()
        with open(products_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
    print(f'Downloaded completed for {name}')
    return

def download_table_products(table, directory, chunk_size=8192):
    """Download WALLABY products from ADQL queried table

    """
    if not os.path.exists(directory):
        os.mkdir(directory)
    print(f'Saving products to {directory}')
    for row in table:
        name = row['source_name']
        products_filename = os.path.join(directory, f'{name}.tar')
        download_products(row, products_filename, chunk_size)
    print('Downloads complete')
    return

In [9]:
# Write output products for a source

download_table_products(table[0:5], tag_name)

Saving products to NGC 5044 DR1
Downloaded completed for WALLABY J134432-244925
Downloaded completed for WALLABY J134414-231547
Downloaded completed for WALLABY J134002-252831
Downloaded completed for WALLABY J134321-251560
Downloaded completed for WALLABY J134255-250726
Downloads complete


---

# Kinematic models

<span style="font-weight: bold; color: #FF0000;">⚠ To be updated</span>

This notebook also allows users to download the kinematic model table and products. First, we select the kinematic models by tag and retrieve the table.

In [10]:
# Select the list of available team release tags

query= """SELECT DISTINCT team_release_kin
        FROM wallaby.kinematic_model """
result = tap.search(query)
result

<Table length=12>
     team_release_kin    
          object         
-------------------------
          NGC5044 Kin TR3
         NGC 5044 Kin TR2
         NGC 5044 Kin TR1
            Hydra Kin TR1
    Vela High-Res Kin TR1
            Hydra Kin TR2
NGC 4808 High-Res Kin TR1
            Norma Kin TR1
         NGC 4636 Kin TR1
             Vela Kin TR1
NGC 5044 High-Res Kin TR1
         NGC 4808 Kin TR1

We can then select the kinematic models corresponding to a specific release key

In [11]:
#   Set the kinematic tag desired
kin_tag="NGC5044 Kin TR3"

#   The generic query
query= """SELECT d.*
        FROM wallaby.kinematic_model d
        WHERE d.team_release_kin IN ('Internal Data Release', '$TAG_NAME')"""
query = query.replace('$TAG_NAME', kin_tag)
#    Run the tap query
result = tap.search(query)
#  Get the resulting table
kin_table = result.to_table()
#  Print off the table
kin_table[0:2]

print(type(kin_table))

<class 'astropy.table.table.Table'>


Once the table is put together, it can be downloaded in an XML format.

In [12]:
# Download catalog table

votable = from_table(kin_table)
votable_filename = f'%s_catalogue.xml' % kin_tag.replace(' ', '_')
votable.to_xml(votable_filename)



We can do a similar thing for the 3KIDNAS tables

In [13]:
kin_tag="NGC 5044 E KinTR3"

In [14]:
#   The generic query
query= """SELECT d.*
        FROM wallaby.kinematic_model_3kidnas d
        WHERE d.team_release_kin IN ('Internal Data Release', '$TAG_NAME')"""
query = query.replace('$TAG_NAME', kin_tag)
#    Run the tap query
result = tap.search(query)
#  Get the resulting table
kin_table_3kidnas = result.to_table()
#  Print off the table
kin_table_3kidnas[0:2]

id,team_release,team_release_kin,vsys_model,e_vsys_model,x_model,e_x_model,y_model,e_y_model,ra_model,e_ra_model,dec_model,e_dec_model,inc_model,e_inc_model,pa_model,e_pa_model,pa_model_g,e_pa_model_g,vdisp_model,e_vdisp_model,rad,vrot_model,e_vrot_model,rad_sd,sd_model,e_sd_model,sdmethodflag,rhi_flag,rhi_as,rhi_low_as,rhi_high_as,dist_model,rhi_kpc,rhi_low_kpc,rhi_high_kpc,vhi_flag,vhi,e_vhi,kflag,kinver
,,,km / s,km / s,pix,pix,pix,pix,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,km / s,km / s,arcsec,km / s,km / s,arcsec,msol / pc2,msol / pc2,,,arcsec,arcsec,arcsec,arcsec,kpc,kpc,kpc,,km / s,km / s,,
int64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,int32,int32,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,int32,object
312,NGC 5044 DR3,NGC 5044 E KinTR3,1470.1,0.6,32.41,0.1802,28.08,0.1327,196.576722,0.000312,-17.511683,0.000221,43.3,3.9,297.1,1.7,296.1,1.7,9.6,0.9,"[7.48, 22.45, 37.42, 52.39, 67.35, 82.32]","[38.0, 30.4, 34.2, 51.5, 53.8, 52.0]","[13.6, 15.6, 8.1, 6.1, 4.2, 7.3]","[7.48, 22.45, 37.42, 52.39, 67.35, 82.32]","[9.2, 6.6, 4.9, 4.1, 3.4, 2.6]","[0.77, 0.89, 0.99, 0.64, 0.42, 0.42]",0,-1,--,--,--,21.00142857142857,--,--,--,-1,--,--,0,3KIDNAS V1
313,NGC 5044 DR3,NGC 5044 E KinTR3,2757.8,1.2,22.3,0.0532,35.29,0.123,198.27106,9.5e-05,-19.976592,0.000205,81.2,1.3,359.7,0.5,359.2,0.5,13.2,4.2,"[7.48, 22.45, 37.42, 52.39, 67.35, 82.32, 97.29, 112.25]","[132.2, 167.5, 131.2, 140.1, 138.9, 150.6, 147.0, 144.2]","[16.5, 49.1, 18.7, 8.8, 10.4, 6.2, 7.3, 11.8]","[7.48, 22.45, 37.42, 52.39, 67.35, 82.32, 97.29, 112.25]","[9.1, 8.0, 6.1, 5.6, 3.5, 2.8, 1.6, 1.5]","[0.75, 1.32, 0.82, 0.84, 0.62, 0.61, 0.93, 0.55]",0,-1,--,--,--,39.39714285714286,--,--,--,-1,--,--,0,3KIDNAS V1


In [16]:
# Download catalog table

votable = from_table(kin_table_3kidnas)
votable_filename = f'%s_catalogue.xml' % kin_tag.replace(' ', '_')
votable.to_xml(votable_filename)


csvtable_filename = f'%s_catalogue.csv' % kin_tag.replace(' ', '_')
kin_table_3kidnas.write(csvtable_filename,overwrite=True,format='csv')


Then we can write the table as a `.fits` file and all products associated with the selected models. As with the source finding table, it will be necessary to replace the object columns with strings.